# Classifing audio data with convolutional neural networks

<br/>
by Eduardo Garcia Rajo @ 2019<br/>
<br/>
This notebook if part of my project "Urban sounds classification with Covnolutional Neural Networks" on my Githubat: https://github.com/GorillaBus/urban-audio-classifier.<br/>
<br/>
Licensed under the GNU LESSER GENERAL PUBLIC LICENSE Version 3, 29 June 2007<br/>
<br/>

## Augmented data pre-processing

MFCC Coefficients extraction from augmented dataset.<br/>
<br/>

### Setup
Pay attention to the very simple path variables configured in this section.

In [1]:
# Required libraries
import sys
import os
import IPython as IP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# Set your path to the dataset
us8k_path = os.path.abspath('./UrbanSound8K')
audio_path = os.path.join(us8k_path, 'audio')
augmented_path = os.path.abspath('augmented')

metadata_augmented_path = os.path.abspath('data/augmented-data.csv')

#### Helper functions

In [3]:
# Helper: Generates MFCC coefficients with Librosa 
def get_mfcc(filename, mfcc_max_padding=0):
    try:
        audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        # Should we require padding
        pad_width = mfcc_max_padding - mfccs.shape[1]
        if (mfcc_max_padding > 0 & mfccs.shape[1] < mfcc_max_padding):
            pad_width = mfcc_max_padding - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
        print("Error parsing wavefile: ", filename)
        return None 
    return mfccs

#### Load metadata

In [4]:
# Load the metadata from the generated CSV
metadata = pd.read_csv(metadata_augmented_path)

# Examine dataframe
print("Metadata length:", len(metadata))
metadata.tail()

Metadata length: 104784


,augment,class,class_id,file,fold
104779,pitch_2.5,gun_shot,6,159705-6-0-0.wav,7
104780,pitch_2.5,gun_shot,6,135527-6-14-9.wav,7
104781,pitch_2.5,gun_shot,6,159708-6-2-0.wav,7
104782,pitch_2.5,gun_shot,6,159708-6-6-0.wav,7
104783,pitch_2.5,gun_shot,6,148827-6-0-0.wav,7


#### Full dataset MFCC extraction

In [5]:
# Iterate through all audio files and extract MFCC
features = []
labels = []
frames_max = 0
counter = 0
total_samples = len(metadata)

for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(audio_path), 'fold' + str(row["fold"]), str(row["file"]))
    
    if (row["augment"] == 'none'):
        file_path = os.path.join(os.path.abspath(audio_path), 'fold' + str(row["fold"]), str(row["file"]))
    else:
        file_path = os.path.join(os.path.abspath(augmented_path), 'fold' + str(row["fold"]), str(row["augment"]), str(row["file"]))
    

    # Extract MFCC coefficients from current file
    mfccs = get_mfcc(file_path)
    

    # Add row (feature / label)
    features.append(mfccs)
    labels.append(row["class"])

    # Update frames maximum
    num_frames = mfccs.shape[1]
    if (num_frames > frames_max):
        frames_max = num_frames

    # Notify update every N files
    if (counter == 500):
        print("Status: {}/{}".format(index+1, total_samples))
        counter = 0

    
    counter += 1
    
print("Finished: {}/{}".format(index, total_samples))
print("Max frame count: {}".format(frames_max))

Status: 501/104784
Status: 1001/104784
Status: 1501/104784
Status: 2001/104784
Status: 2501/104784
Status: 3001/104784
Status: 3501/104784
Status: 4001/104784
Status: 4501/104784
Status: 5001/104784
Status: 5501/104784
Status: 6001/104784
Status: 6501/104784
Status: 7001/104784
Status: 7501/104784
Status: 8001/104784
Status: 8501/104784
Status: 9001/104784
Status: 9501/104784
Status: 10001/104784
Status: 10501/104784
Status: 11001/104784
Status: 11501/104784
Status: 12001/104784
Status: 12501/104784
Status: 13001/104784
Status: 13501/104784
Status: 14001/104784
Status: 14501/104784
Status: 15001/104784
Status: 15501/104784
Status: 16001/104784
Status: 16501/104784
Status: 17001/104784
Status: 17501/104784
Status: 18001/104784
Status: 18501/104784
Status: 19001/104784
Status: 19501/104784
Status: 20001/104784
Status: 20501/104784
Status: 21001/104784
Status: 21501/104784
Status: 22001/104784
Status: 22501/104784
Status: 23001/104784
Status: 23501/104784
Status: 24001/104784
Status: 2450

#### Add padding for a consistent shape

In [6]:
padded = []

# Add padding
mfcc_max_padding = frames_max
for i in range(len(features)):
    size = len(features[i][0])
    if (size < mfcc_max_padding):
        pad_width = mfcc_max_padding - size
        px = np.pad(features[i], 
                    pad_width=((0, 0), (0, pad_width)), 
                    mode='constant', 
                    constant_values=(0,))
    
    padded.append(px)

#### Save X and y

In [7]:
# Convert features (X) and labels (y) to Numpy arrays

X = np.array(padded)
y = np.array(labels)

# Optionally save the features to disk
np.save("data/X-augmented", X)
np.save("data/y-augmented", y)

In [8]:
# Verify shapes
print("Raw features length: {}".format(len(features)))
print("Padded features length: {}".format(len(padded)))
print("Feature labels length: {}".format(len(features)))
print("X: {}, y: {}".format(X.shape, y.shape))


Raw features length: 104784
Padded features length: 104784
Feature labels length: 104784
X: (104784, 40, 215), y: (104784,)
